# 1. pre-process dict

## 1.1 Read dict

In [1]:
#read the dictionary
dic = []
with open("dict.txt","r") as dic_fr:
    dic = dic_fr.readlines()
for i in range(len(dic)):
    d = dic[i]
    dic[i] = d.strip()

## 1.2 remove '-' and blank line and stopwords in dict

In [2]:
tmp_dict = []
for i in range(len(dic)):
    if '-' in dic[i]:# remove '-'
        tmp_dict.append( dic[i].replace("-"," ") )
    else:
        tmp_dict.append( dic[i] )
dic = tmp_dict

In [3]:
tmp_dict = []
for i in range(len(dic)):
    if '' != dic[i]:# remove ''
        tmp_dict.append( dic[i] )
dic = tmp_dict

In [4]:
# remove stopwords
import nltk
english_stopwords = nltk.corpus.stopwords.words("english")
tmp_dict = []
for i in range(len(dic)):
    words = nltk.word_tokenize(dic[i])
    tmp = []
    for w in words:
        if w not in english_stopwords:
            tmp.append(w)
    tmp_dict.append(' '.join(tmp)) 
dic = tmp_dict

In [5]:
len(dic)

512

## 1.3 dic capital letter 2 lower

In [6]:
for i in range(len(dic)):
    dic_phrase_word_list = dic[i].split()
    for i in range(len(dic_phrase_word_list)):
        word = dic_phrase_word_list[i]
        capital_letter_count = 0
        for letter in word:
            if not letter.islower():
                capital_letter_count+=1
        if capital_letter_count>2:
            pass
        else:
            dic_phrase_word_list[i] = word.lower()
    dic[i] = ' '.join(dic_phrase_word_list)

## 2. pre-process text

## 2.1 Read text as list

In [8]:
%%time
#read dict data
text_abstract_dictdata = []
text_title_dictdata = []
with open ("result_chrome_dict.txt",'r',encoding = "UTF-8") as fr:
    lines = fr.readlines()
    for line in lines:
        tmp = line.split("abstract:")# abstract only
        if(len(tmp)>1):
            line_split = tmp[1]
            line_abstract_split = line_split.strip()
            text_abstract_dictdata.append(line_abstract_split)
            text_title_split_tmp = tmp[0].strip("title:")
            text_title_split = text_title_split_tmp.strip("===")
            text_title_dictdata.append(text_title_split)
            #text_title = text_title.append(text_title.strip("title:"))    

Wall time: 276 ms


In [10]:
%%time
#dic data --->title+abstract
text_list_dictdata = []
for i in range(len(text_title_dictdata)):
    for j in range(len(text_abstract_dictdata)):
        if i == j:
            text_list_tmp = text_title_dictdata[i]+ " " + text_abstract_dictdata[j]
            text_list_dictdata.append(text_list_tmp)

Wall time: 53.1 s


In [11]:
#read radar data
import json
import re
with open('result_chrome_1.json','r',encoding = 'utf-8') as fr:
    lines = fr.readlines()
rep = re.compile("\xA9.*")

result_abstract_radardata = []
for line in lines:
    try:
        line = json.loads(line)
        abstract = rep.sub("", line['abstract']).strip()
        result_abstract_radardata.append(abstract)
    except Exception as e:
        pass

result_title_radardata = []
for line in lines:
    try:
        jdata = json.loads(line)
        result_title_radardata.append(jdata['title'])
    except Exception as e:
        pass

In [12]:
%%time
#radar data --->title+abstract
text_list_radardata = []
for i in range(len(result_title_radardata)):
    for j in range(len(result_abstract_radardata)):
        if i == j:
            text_tmp = result_title_radardata[i]+ " " + result_abstract_radardata[j]
            text_list_radardata.append(text_tmp)

Wall time: 50.5 s


In [14]:
# get all data named text_list
text_list = text_list_dictdata + text_list_radardata

In [15]:
len(text_list)

43117

## 2.2 Remove the repeated text

In [16]:
text_list = list(set(text_list))
len(text_list)

39428

In [64]:
#------------test code---------------------
tmp_txt = text_list
tmp_txt = " ".join(tmp_txt)
count = 0
for i in range(len(dic)):
    text_split_by_phrase = tmp_txt.split(dic[i])
    count += len(text_split_by_phrase)-1
    count_tmp = len(text_split_by_phrase)-1
    #print('{0}: count={1}'.format(i, count_tmp))
count

31853

## 2.3 Remove "this paper","in this paper"

In [17]:
useless = ["this paper","in this paper","This paper","In this paper", '[无可用摘要]','0','1','2','3','4','5','6','7','8','9','<','>','[',']','``',"''",'.',"'",'``','ghz','GHz',"''",]
for i in range(len(useless)):
    for j in range(len(text_list)):
        if useless[i] in text_list[j]:
            text_list[j] = text_list[j].replace(useless[i]," ")

In [66]:
#------------test code---------------------
tmp_txt = text_list
tmp_txt = " ".join(tmp_txt)
count = 0
for i in range(len(dic)):
    text_split_by_phrase = tmp_txt.split(dic[i])
    count += len(text_split_by_phrase)-1
    #count_tmp = len(text_split_by_phrase)-1
    #print('{0}: count={1}'.format(i, count_tmp))
count

31853

In [18]:
len(text_list)

39428

## 2.4 remove part capital letter in text

In [19]:
%%time
tmp_text_list = []
for txt in text_list:
    capital_letter_count = 0
    sum_letter_count = 0
    for a in txt:
        if not a.islower():
            capital_letter_count+=1
        sum_letter_count+=1
    if(capital_letter_count/sum_letter_count<0.9):
        tmp_text_list.append(txt)
text_list = tmp_text_list

Wall time: 10.3 s


In [20]:
len(text_list)

39387

In [70]:
#------------test code---------------------
tmp_txt = text_list
tmp_txt = " ".join(tmp_txt)
count = 0
for i in range(len(dic)):
    text_split_by_phrase = tmp_txt.split(dic[i])
    count += len(text_split_by_phrase)-1
    #count_tmp = len(text_split_by_phrase)-1
    #print('{0}: count={1}'.format(i, count_tmp))
count

31853

## 2.5 tokenize

In [21]:
text = ' '.join(text_list)#将文本变为一行

In [22]:
len(text)

45625993

In [23]:
%%time
import nltk
word_list = nltk.word_tokenize(text)#分词

Wall time: 36.2 s


In [25]:
len(word_list)

7084581

In [75]:
#------------test code---------------------
tmp_txt = text_list
tmp_txt = " ".join(tmp_txt)
count = 0
for i in range(len(dic)):
    text_split_by_phrase = tmp_txt.split(dic[i])
    count += len(text_split_by_phrase)-1
    #count_tmp = len(text_split_by_phrase)-1
    #print('{0}: count={1}'.format(i, count_tmp))
count

31853

## 2.6 remove punctuation in text

In [26]:
%%time
from string import punctuation
tmp = []
for item in word_list:
    if (item not in punctuation):
        tmp.append(item)        
word_list = tmp

Wall time: 1.93 s


In [27]:
len(word_list)

6622496

In [78]:
#------------test code---------------------
tmp_txt = ' '.join(word_list)
count = 0
for i in range(len(dic)):
    text_split_by_phrase = tmp_txt.split(dic[i])
    count += len(text_split_by_phrase)-1
    #count_tmp = len(text_split_by_phrase)-1
    #print('{0}: count={1}'.format(i, count_tmp))
count

32381

## 2.7 remove '-' and blank line in text

In [28]:
tmp_txt = []
for i in range(len(word_list)):
    if '-' in word_list[i]:
        tmp_txt.append( word_list[i].replace("-"," ") )
    else:
        tmp_txt.append( word_list[i] )
word_list = tmp_txt

In [29]:
len(word_list)

6622496

In [81]:
#------------test code---------------------
tmp_txt = ' '.join(word_list)
count = 0
for i in range(len(dic)):
    text_split_by_phrase = tmp_txt.split(dic[i])
    count += len(text_split_by_phrase)-1
    #count_tmp = len(text_split_by_phrase)-1
    #print('{0}: count={1}'.format(i, count_tmp))
count

34714

## 2.8 remove stop words in text

In [30]:
tmp_txt = []
english_stopwords = nltk.corpus.stopwords.words("english")
for i in range(len(word_list)):
    if word_list[i] not in english_stopwords:
           tmp_txt.append(word_list[i]) 
word_list = tmp_txt

In [83]:
#------------test code---------------------
tmp_txt = ' '.join(word_list)
count = 0
for i in range(len(dic)):
    text_split_by_phrase = tmp_txt.split(dic[i])
    count += len(text_split_by_phrase)-1
    #count_tmp = len(text_split_by_phrase)-1
    #print('{0}: count={1}'.format(i, count_tmp))
count

35291

In [31]:
len(word_list)

4420662

## 2.9 capital 2 lower


In [32]:
#text 
for i in range(len(word_list)):
    word = word_list[i]
    capital_letter_count = 0
    for letter in word:
        if not letter.islower():
            capital_letter_count+=1      
    if capital_letter_count>2:
        pass
    else:
        word_list[i] = word.lower()

In [33]:
len(word_list)

4420662

In [34]:
#------------test code---------------------
tmp_txt = ' '.join(word_list)
count = 0
for i in range(len(dic)):
    text_split_by_phrase = tmp_txt.split(dic[i])
    count += len(text_split_by_phrase)-1
    #count_tmp = len(text_split_by_phrase)-1
    #print('{0}: count={1}'.format(i, count_tmp))
count

46487

## 2.10 stemming

### 2.10.1 use snowball stemming methods in NLTK for our corpus

In [35]:
tmp_word_list_1 = word_list

In [36]:
%%time
# text stemming by EnglishStemmer
from nltk.stem.snowball import EnglishStemmer
st = EnglishStemmer()

for i in range(len(tmp_word_list_1)):
    words = tmp_word_list_1[i].split(' ')
    wordslist = []
    for w in words:
        wordslist.append(st.stem(w))
    
    tmp_word_list_1[i] = ' '.join(wordslist)

Wall time: 1min 47s


In [37]:
%%time
# dic stemming by EnglishStemmer
from nltk.stem.snowball import EnglishStemmer
st = EnglishStemmer()
tmp_dic_1 = dic
for i in range(len(tmp_dic_1)):
    dic_phrase_word_list = tmp_dic_1[i].split(' ')
    for j in range(len(dic_phrase_word_list)):
        word = dic_phrase_word_list[j]
        dic_phrase_word_list[j] = st.stem(word)
    tmp_dic_1[i] = ' '.join(dic_phrase_word_list)

Wall time: 39 ms


In [38]:
#------------test code---------------------
tmp_txt_1 = ' '.join(tmp_word_list_1)
count = 0
for i in range(len(tmp_dic_1)):
    text_split_by_phrase = tmp_txt_1.split(dic[i])
    count += len(text_split_by_phrase)-1
    #count_tmp = len(text_split_by_phrase)-1
    #print('{0}: count={1}'.format(i, count_tmp))
count

64741

### 2.10.2 use wordnet stemming methods in NLTK for our corpus

In [39]:
tmp_word_list_2 = word_list

In [40]:
%%time
#text stemming by WordNetLemmatizer
from nltk.stem.wordnet import WordNetLemmatizer
st = WordNetLemmatizer()
for i in range(len(tmp_word_list_2)):
    words = tmp_word_list_2[i].split(' ')
    wordslist = []
    for w in words:
        wordslist.append(st.lemmatize(w))
    
    tmp_word_list_2[i] = ' '.join(wordslist)

Wall time: 40 s


In [41]:
%%time
# dic stemming by WordNetLemmatizer
from nltk.stem.wordnet import WordNetLemmatizer
st = WordNetLemmatizer()
tmp_dic_2 = dic
for i in range(len(tmp_dic_2)):
    dic_phrase_word_list = tmp_dic_2[i].split(' ')
    for j in range(len(dic_phrase_word_list)):
        word = dic_phrase_word_list[j]
        dic_phrase_word_list[j] = st.lemmatize(word)
    tmp_dic_2[i] = ' '.join(dic_phrase_word_list)

Wall time: 11 ms


In [42]:
#------------test code---------------------
tmp_txt_2 = ' '.join(tmp_word_list_2)
count = 0
for i in range(len(tmp_dic_2)):
    text_split_by_phrase = tmp_txt_2.split(dic[i])
    count += len(text_split_by_phrase)-1
    #count_tmp = len(text_split_by_phrase)-1
    #print('{0}: count={1}'.format(i, count_tmp))
count

64712

## Finally, we choose the snowball stemmer.

# 3.Dump the dataset by pickle

In [43]:
import pickle

pickle.dump(tmp_txt_1, open('txt_preprocessed.p', 'wb'))
pickle.dump(tmp_dic_1, open('list_dic_preprocessed.p', 'wb'))